# Homework 4: Language Modelling in Hangman

Student Name: Zihang Su

Student ID: 710118

Python version used: Python 3

## General info

<b>Due date</b>: 11pm, Wednesday May 2nd

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character level language models, ranging from unigram approaches to higher over n-gram models. Your objective is to create an automatic player which makes the fewest mistakes.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## The Hangman Game

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game, and a method allowing interactive play. 

In [1]:
# allowing better python 2 & python 3 compatibility 
from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed, **kwargs):
    """
    simple function for manual play
    """
    print('Enter your guess:')
    try:
        return raw_input().lower().strip() # python 3
    except NameError:
        return input().lower().strip() # python 2

You can play the game interactively using the following command:

In [2]:
hangman('integer', human, 26, False)

Enter your guess:
i
Enter your guess:
n
Enter your guess:
t
Enter your guess:
e
Enter your guess:
g
Enter your guess:
r


0

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the method. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown`, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(0.5 mark)

In [2]:
from nltk.corpus import brown
import numpy as np
import re

word_types = set()
for sent in brown.sents():
    for word in sent:
        if re.match(r'^[a-zA-Z]+$', word):
            word_types.add(word.lower())

number_of_testing_word = 1000
word_types = list(word_types)
np.random.shuffle(word_types)
training_set = word_types[0:-number_of_testing_word]
test_set = word_types[-number_of_testing_word:]

print('training set:  ' + str(len(training_set)))
print('test set:      ' + str(len(test_set)))

training set:  39234
test set:      1000


<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To measure the performance of this (and later) techiques, implement a method that measures the average number of mistakes made by this technique over all the words in the `test_set`. You will want to turn off the printouts for this, using the `verbose=False` option, and increase the cap on the game length to `max_mistakes=26`. Print the average number of mistakes for the random AI, which will become a baseline for the following steps.

(1 mark)

In [3]:
alphabetic = list('abcdefghijklmnopqrstuvwxyz')

def first_AI(mask, guessed, **guesser_args):
    my_guess = np.random.choice(alphabetic)
    while (my_guess in guessed):
        my_guess = np.random.choice(alphabetic)
    return my_guess

def average_length(iteration, guesser):
    count = 0
    for i in range(iteration):
        mistake = hangman(np.random.choice(test_set), guesser, 26, False)
        count += mistake
    return count/iteration

number_of_iteration = 1000
average = average_length(number_of_iteration, first_AI)
print('Average mistakes for random AI approach:   ' + str(average))

Average mistakes for random AI approach:   16.641


**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guess function that returns the character with the highest probability, after aggregating (summing) the probability of each blank character in the secret word. Print the average number of mistakes the unigram method makes over the test set. Remember to exclude already guessed characters, and use the same evaluation method as above, so the results are comparable. (Hint: it should be much lower than for random).

(1 mark)

In [4]:
from collections import Counter
from numpy.random import choice

def unigram_count():  # can input as a corpus
    character_counts = Counter()
    
    for word in training_set:
        for character in word:
            character_counts[character] += 1
    return character_counts

# We must have unigram_counts before we run this function.
def unigram(mask, guessed, **guesser_args):
    max_value = 0
    for c in alphabetic:
        if (c not in guessed) and (unigram_counts[c] >= max_value):
            max_value = unigram_counts[c]
            character = c
    return character


unigram_counts = unigram_count()
average_with_length = average_length(number_of_iteration, unigram)
print('Average mistakes for unigram:    ' + str(average_with_length))

Average mistakes for unigram:    10.325


**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. Your job now is to incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having *different* unigram models for each length of word. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. Create another AI guessing function using this new model, and print its test performance.   

(0.5 marks)

In [5]:
from collections import defaultdict

def unigram_count_with_length():  # can input as a corpus
    character_counts = defaultdict(Counter)
    
    for word in training_set:
        length = len(word)
        for character in word:
            character_counts[length][character] += 1
            
    return character_counts

# We must have length_uni_count before we run this function.
def length_unigram(mask, guessed, **guesser_args):
    length = len(mask)
    character = ''
    if length in length_uni_count.keys():
        max_value = 0
        for c in alphabetic:
            if (c not in guessed) and (length_uni_count[length][c] >= max_value):
                max_value = length_uni_count[length][c]
                character = c
    else:
        max_value = 0
        for c in alphabetic:
            if (c not in guessed) and (unigram_counts[c] >= max_value):
                max_value = unigram_counts[c]
                character = c
    return character


def compare_average_length(iteration, guesser1, guesser2):
    count1 = 0
    count2 = 0
    for i in range(iteration):
        word = np.random.choice(test_set)
        mistake1 = hangman(word, guesser1, 26, False)
        mistake2 = hangman(word, guesser2, 26, False)
        count1 += mistake1
        count2 += mistake2
    return count1/iteration, count2/iteration

unigram_counts = unigram_count()
length_uni_count = unigram_count_with_length()
average1, average2 = compare_average_length(number_of_iteration, unigram, length_unigram)
print('without length:  ' + str(average1))
print('with length:     ' + str(average2))

without length:  10.604
with length:     10.516


**Instructions:** Now for the main challenge, using a *ngram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

Your job is to develop a *ngram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with sentinel symbols.) You should use linear interpolation to smooth between the higher order and lower order models, and you will have to decide how to weight each component. 

Your guessing AI algorithm should apply your language model to each blank position in the secret word by using as much of the left context as is known. E.g., in `_ e c _ e _ _` we know the full left context for the first blank (context=start of word), we have a context of two characters for the second blank (context=ec), one character for the second last blank (context=e), and no known context for the last one. If we were using a *n=3* order model, we would be able to apply it to the first and second blanks, but would only be able to use the bigram or unigram distributions for the subsequent blanks. As with the unigram model, you should sum over the probability distributions for each blank to find the expected count for each character type, then select the  character with the highest expected count.

Implement the ngram method for *n=3,4* and *5* and evaluate each of these three models over the test set. Do you see any improvement over the unigram methods above?

(2 marks)

In [6]:
import numpy as np

def unigram_count_with_start_end():  # can input as a corpus
    character_counts = Counter()
    
    for word in training_set:
        word = [c.lower() for c in word]
        for character in word:
            character_counts[character] += 1
    return character_counts


def bigram_count():
    character_counts = defaultdict(Counter)
    for word in training_set:
        word = ["<s1>"] + [c.lower() for c in word] + ["</s1>"]
        bigram_list = zip(word[:-1], word[1:])
        for bigram in bigram_list:
            first, second = bigram
            character_counts[first][second] += 1
    
    return character_counts

def trigram_count():
    character_counts = defaultdict(lambda: defaultdict(Counter))
    for word in training_set:
        word = ["<s1>", "<s2>"] + [c.lower() for c in word] + ["</s1>", "</s2>"]
        trigram_list = zip(word[:-2], word[1:-1], word[2:])
        for trigram in trigram_list:
            first, second, third = trigram
            character_counts[first][second][third] += 1
    
    return character_counts

def four_gram_count():
    character_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(Counter)))
    for word in training_set:
        word = ["<s1>", "<s2>", "<s3>"] + [c.lower() for c in word] + ["</s1>", "</s2>", "</s3>"]
        four_gram_list = zip(word[:-3], word[1:-2], word[2:-1], word[3:])
        for four_gram in four_gram_list:
            first, second, third, fourth = four_gram
            character_counts[first][second][third][fourth] += 1
    
    return character_counts

def five_gram_count():
    character_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(Counter))))
    for word in training_set:
        word = ["<s1>", "<s2>", "<s3>", "<s4>"] + [c.lower() for c in word] + ["</s1>", "</s2>", "</s3>", "</s4>"]
        five_gram_list = zip(word[:-4], word[1:-3], word[2:-2], word[3:-1], word[4:])
        for five_gram in five_gram_list:
            first, second, third, fourth, fifth = five_gram
            character_counts[first][second][third][fourth][fifth] += 1
    
    return character_counts

ngram_count_1 = unigram_count_with_start_end()
ngram_count_2 = bigram_count()
ngram_count_3 = trigram_count()
ngram_count_4 = four_gram_count()
ngram_count_5 = five_gram_count()

def unigram_prob(word):
    denominator = sum(ngram_count_1.values())
    if denominator != 0:
        prob = ngram_count_1[word]/denominator
    else:
        prob = 0.0
    return prob

def bigram_prob(prev1, word):   # input are two words
    denominator = sum(ngram_count_2[prev1].values())
    if denominator != 0:
        prob = 0.5*(ngram_count_2[prev1][word]/denominator)
        + 0.5*(unigram_prob(word))
    else:
        prob = 0.5*(unigram_prob(word))
    return prob

def trigram_prob(prev2, prev1, word):
    denominator = sum(ngram_count_3[prev2][prev1].values())
    if denominator != 0:
        prob = 0.33*(ngram_count_3[prev2][prev1][word]/denominator)
        + 0.67*(bigram_prob(prev1, word))
    else:
        prob = 0.67*(bigram_prob(prev1, word))
    return prob

def four_gram_prob(prev3, prev2, prev1, word):
    denominator = sum(ngram_count_4[prev3][prev2][prev1].values())
    if denominator != 0:
        prob = 0.25*(ngram_count_4[prev3][prev2][prev1][word]/denominator)
        + 0.75*(trigram_prob(prev2, prev1, word))
    else:
        prob = 0.75*(trigram_prob(prev2, prev1, word))
    return prob

def five_gram_prob(prev4, prev3, prev2, prev1, word):
    denominator = sum(ngram_count_5[prev4][prev3][prev2][prev1].values())
    if denominator != 0:
        prob = 0.2*(ngram_count_5[prev4][prev3][prev2][prev1][word]/denominator)
        + 0.8*(four_gram_prob(prev3, prev2, prev1, word))
    else:
        prob = 0.8*(four_gram_prob(prev3, prev2, prev1, word))
    return prob

def bigram(mask, guessed, **guesser_args):
    mask = ["<s1>"] + mask + ["</s1>"]
    length = len(mask)
    prob_matrix = np.zeros((length, 26))

    for i in range(1,length-1):
        if mask[i] == '_':
            prev_1 = mask[i-1]
            for j in range(26):
                current = alphabetic[j]
                prob_matrix[i][j] = bigram_prob(prev_1, current)

    max_value = 0
    for m in range(26):
        score = sum(prob_matrix[:, m])
        c = alphabetic[m]
        if (c not in guessed) and (score >= max_value):
            max_value = score
            character = c
    return character


def trigram(mask, guessed, **guesser_args):
    mask = ["<s1>", "<s2>"] + mask + ["</s1>", "</s2>"]
    length = len(mask)
    prob_matrix = np.zeros((length, 26))

    for i in range(2,length-2):
        if mask[i] == '_':
            prev_1 = mask[i-1]
            prev_2 = mask[i-2]
            for j in range(26):
                current = alphabetic[j]
                prob_matrix[i][j] = trigram_prob(prev_2, prev_1, current)
                
    max_value = 0
    for m in range(26):
        score = sum(prob_matrix[:, m])
        c = alphabetic[m]
        if (c not in guessed) and (score >= max_value):
            max_value = score
            character = c
    return character


def four_gram(mask, guessed, **guesser_args):
    mask = ["<s1>", "<s2>", "<s3>"] + mask + ["</s1>", "</s2>", "</s3>"]
    length = len(mask)
    prob_matrix = np.zeros((length, 26))

    for i in range(3,length-3):
        if mask[i] == '_':
            prev_1 = mask[i-1]
            prev_2 = mask[i-2]
            prev_3 = mask[i-3]
            for j in range(26):
                current = alphabetic[j]
                prob_matrix[i][j] = four_gram_prob(prev_3, prev_2, prev_1, current)
                
    max_value = 0
    for m in range(26):
        score = sum(prob_matrix[:, m])
        c = alphabetic[m]
        if (c not in guessed) and (score >= max_value):
            max_value = score
            character = c
    return character

def five_gram(mask, guessed, **guesser_args):
    mask = ["<s1>", "<s2>", "<s3>", "<s4>"] + mask + ["</s1>", "</s2>", "</s3>", "</s4>"]
    length = len(mask)
    prob_matrix = np.zeros((length, 26))

    for i in range(4,length-4):
        if mask[i] == '_':
            prev_1 = mask[i-1]
            prev_2 = mask[i-2]
            prev_3 = mask[i-3]
            prev_4 = mask[i-4]
            for j in range(26):
                current = alphabetic[j]
                prob_matrix[i][j] = five_gram_prob(prev_4, prev_3, prev_2, prev_1, current)
                
    max_value = 0
    for m in range(26):
        score = sum(prob_matrix[:, m])
        c = alphabetic[m]
        if (c not in guessed) and (score >= max_value):
            max_value = score
            character = c
    return character


ngram_3 = average_length(number_of_iteration, trigram)
ngram_4 = average_length(number_of_iteration, four_gram)
ngram_5 = average_length(number_of_iteration, five_gram)

print('Average mistakes(3-gram):  ' + str(ngram_3))
print('Average mistakes(4-gram):  ' + str(ngram_4))
print('Average mistakes(5-gram):  ' + str(ngram_5))

Average mistakes(3-gram):  8.175
Average mistakes(4-gram):  7.869
Average mistakes(5-gram):  7.745


## Bonus: Improving the AI

**Instructions:** To get the extra credit, you should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word and the parts that are known, fancier smoothing methods, using backwards ngram models, or a fancier inference algorithm like forward-back using a HMM. Ensure you report the test performance of your method.

You will be marked based on the ambition of your approach and on its accuracy. If you have tried some truly spectacular method but it didn't really work, then please include your implementation and an explanation, which will still attract marks for ambition.

(1 bonus mark) 